In [ ]:
pip install fiftyone


In [ ]:
pip install opencv-python-headless==4.5.2.52

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

NumExpr defaulting to 2 threads.
Migrating database to v0.15.0.1


In [ ]:
############## This to download data from openimage ----trainning don't run it ###############

dataset = fo.zoo.load_zoo_dataset(
              "open-images-v6",
              split="train",
              label_types=["detections"],
              classes=['Jellyfish', 'Dolphin', 'Starfish', 'Shellfish', 'Turtle']
          )


 100% |██████|    4.8Gb/4.8Gb [4.0s elapsed, 0s remaining, 1.2Gb/s]       
 100% |█████████████████| 2573/2573 [3.3m elapsed, 0s remaining, 12.2 files/s]      
Dataset info written to '/root/fiftyone/open-images-v6/info.json'
Loading 'open-images-v6' split 'train'
 100% |███████████████| 2573/2573 [16.6s elapsed, 0s remaining, 143.3 samples/s]      
Dataset 'open-images-v6-train' created


In [ ]:
############## This to download data from openimage ----testing don't run it ###############

dataset = fo.zoo.load_zoo_dataset(
              "open-images-v6",
              split="test",
              label_types=["detections"],
              classes=['Jellyfish', 'Dolphin', 'Starfish', 'Shellfish', 'Turtle']
          )


 100% |███████████████████| 469/469 [37.3s elapsed, 0s remaining, 13.1 files/s]      
Dataset info written to '/root/fiftyone/open-images-v6/info.json'
Loading 'open-images-v6' split 'test'
 100% |█████████████████| 469/469 [3.9s elapsed, 0s remaining, 127.8 samples/s]      
Dataset 'open-images-v6-test' created


In [ ]:
############## This to download data from openimage ----validation don't run it ###############
dataset = fo.zoo.load_zoo_dataset(
              "open-images-v6",
              split="validation",
              label_types=["detections"],
              classes=['Jellyfish', 'Dolphin', 'Starfish', 'Shellfish', 'Turtle']
          )


 100% |███████████████████| 172/172 [13.5s elapsed, 0s remaining, 14.3 files/s]      
Dataset info written to '/root/fiftyone/open-images-v6/info.json'
Loading 'open-images-v6' split 'validation'
 100% |█████████████████| 172/172 [1.4s elapsed, 0s remaining, 123.5 samples/s]         
Dataset 'open-images-v6-validation' created


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [ ]:
pip install chitra

In [ ]:
from __future__ import print_function, division
from chitra.image import Chitra
import os
import torch.nn as nn
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
import torch.optim as optim
import xml.etree.ElementTree as ET
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.retinanet import RetinaNet
import warnings
import transforms as T
from torchvision.transforms import transforms as T1
from torchvision.transforms import AutoAugment
from torchvision.transforms import functional as TF
from torchvision.transforms.autoaugment import AutoAugmentPolicy
import utils
from torch.utils.data.sampler import SubsetRandomSampler
from engine import train_one_epoch
from engine import evaluate
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import models
import torch.nn.functional as F
import torch

warnings.filterwarnings("ignore")
plt.ion()

In [ ]:
pip install torchvision -U

In [ ]:
def create_corner_rect(bb, color='white'):
    bb = np.array(bb.detach(), dtype=np.float64)
    return plt.Rectangle((bb[0], bb[1]),bb[2]-bb[0], bb[3]-bb[1] , color=color,
                         fill=False, lw=3)

def show_corner_bb(im, bb, color):
    plt.imshow(im)
    plt.gca().add_patch(create_corner_rect(bb,color))

In [ ]:
data_dir = os.listdir("/content/drive/MyDrive/data/openimages_dataset")
clase = ['Jellyfish', 'Fish', 'Dolphin', 'Starfish', 'Shellfish', 'Turtle']

In [ ]:
Train = pd.read_csv("/content/drive/MyDrive/data/openimages_dataset/train/labels/detections.csv", sep=",")
Train_classes = pd.read_csv("/content/drive/MyDrive/data/openimages_dataset/train/metadata/classes.csv", sep=",")
Train_classes.columns = ['classID', 'classname']
Train_classes = Train_classes[Train_classes['classname'].isin(clase)]
Train = Train[Train['LabelName'].isin(Train_classes['classID'])]

In [ ]:
Test = pd.read_csv("/content/drive/MyDrive/data/openimages_dataset/test/labels/detections.csv", sep=",")
Test_classes = pd.read_csv("/content/drive/MyDrive/data/openimages_dataset/test/metadata/classes.csv", sep=",")
Test_classes.columns = ['classID', 'classname']
Test_classes = Test_classes[Test_classes['classname'].isin(clase)]
Test = Test[Test['LabelName'].isin(Test_classes['classID'])]


Valid  = pd.read_csv("/content/drive/MyDrive/data/openimages_dataset/validation/labels/detections.csv", sep=",")
Valid_classes = pd.read_csv("/content/drive/MyDrive/data/openimages_dataset/validation/metadata/classes.csv", sep=",")
Valid_classes.columns = ['classID', 'classname']
Valid_classes = Valid_classes[Valid_classes['classname'].isin(clase)]
Valid = Valid[Valid['LabelName'].isin(Valid_classes['classID'])]

In [ ]:
def get_transform1(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomPhotometricDistort())
        transforms.append(T.RandomZoomOut())
        transforms.append(T.RandomIoUCrop())
        transforms.append(T.ScaleJitter((224,224)))
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:

class FothomDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms, Train, Train_classes):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.images_names = os.listdir(self.root)
        self.Train = Train
        self.Train_classes =Train_classes
    def __getitem__(self, idx):
        # load images and bounding boxes
        # 1st load image
        img_path = self.images_names[idx]
        img = Image.open(os.path.join(self.root,img_path)).convert("RGB")
        width, height = img.size

        # 2end extract bounding boxes
        example_of_image = self.Train[self.Train['ImageID']==img_path[:-4]]
        boxes = np.array(example_of_image[['XMin','XMax','YMin','YMax']])
        boxes[:,0] = boxes[:,0]*width
        boxes[:,1] = boxes[:,1]*width
        boxes[:,2] = boxes[:,2]*height
        boxes[:,3] = boxes[:,3]*height
        boxes =boxes[:,[0,2,1,3]]
        boxes = boxes.tolist()
        # 3erd extract labels
        labels = []
        for e in range(len(example_of_image)):
          labels.append(clase.index(self.Train_classes[example_of_image['LabelName'].iloc[e]==self.Train_classes['classID']]['classname'].iloc[0]))
        
        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.as_tensor(labels, dtype=torch.int64)
        boxe = []
        for k,elem in enumerate(boxes): 
          resize = Chitra(img,elem,labels[k] )
          resize.resize_image_with_bbox((224, 224))
          if(k == len(boxes)-1):
            img = resize.image
          boxes1 = np.concatenate(resize.bboxes[0][:], axis=None)
          boxe.append(list(boxes1))
        boxes = torch.as_tensor(boxe, dtype=torch.float32)
        '''print(img)
        print(boxes)
        print(labels[0])'''



        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        #target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)
        
        y= []
        y.append(target)
        #bg = T1.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        #img = bg(img)
        return img, y

    def __len__(self):
        return len(self.images_names)
    

In [ ]:
def model_choice(modeltype,num_classes): #choose between the exist model in pytorch modeltype: ['fasterrcnn, mobilenet']
    if(modeltype == 'model_0'):
        model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True)

    if(modeltype == 'model_1'):
        model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
    
    if(modeltype == 'model_2'):
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    if(modeltype == 'model_3'):
        model = torchvision.models.detection.FasterRCNN(pretrained=True)

    num_classes = num_classes  
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [ ]:
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has 5 classes 
num_classes = len(clase)
# use our dataset and defined transformations (our class)
dataset =  FothomDataset('/content/drive/MyDrive/data/openimages_dataset/train/data', get_transform1(train=True),Train,Train_classes)
dataset_valid =  FothomDataset('/content/drive/MyDrive/data/openimages_dataset/validation/data', get_transform1(train=False),Valid,Valid_classes)


# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-4000])

# this function from torch read and prepare the data to be iterable, means data_loader contain, at each iteration, the image and the bb and the labels
data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=1)

data_loader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=1)

# get the model using our helper function 
model = model_choice('model_0',num_classes)

# move model to the right device
model.to(device)

# construct an optimizer

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=1e-3,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=5)


TypeError: ignored

In [ ]:
num_epochs = 1

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_valid, device=device)

In [ ]:
checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
torch.save(checkpoint, 'checkpoint_frcnnm_bn.pth')

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model1 = model_choice('model_0',num_classes)
model1 = model1.to(device)
checkpoint = torch.load('checkpoint_frcnnm_bn.pth')
model1.load_state_dict(checkpoint['state_dict'])
model1.eval()

## **Test **

In [ ]:
dataset_test =  FothomDataset('/content/drive/MyDrive/data/openimages_dataset/test/data', get_transform1(train=False),Test,Test_classes)
data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1)

In [ ]:
evaluate(model1, data_loader_test, device)

#Calculate accuracy

In [ ]:
from sklearn.metrics import accuracy_score
def binary_acc(predicted_label, correct_lablel):
    return (sum([1 for i, j in zip(predicted_label, correct_lablel) if i == j[0]])/len(predicted_label))*100

def val_metrics(model, valid_dl, C=100):
    model.eval()
    correct_lablel=[]
    predicted_label =[]
    i = 0
    for images, targets in valid_dl:
        images = list(image.to(device) for image in images)
        boxes =targets[0]['boxes']
        y_class = targets[0]['labels']
        if(len(boxes==3)):
            #print(boxes)
            element = max(boxes.shape[0],boxes.shape[1])
            boxes = torch.reshape(boxes, (element,boxes.shape[2]))   
            targets[0]['boxes']= boxes
        output = model(images)
        out_bb = output[0]['boxes']
        out_class = output[0]['labels']
        correct_lablel.append(y_class[0])
        try :
            predicted_label.append(out_class[0])
        except :
            predicted_label.append(0)
        
    return predicted_label,correct_lablel

In [ ]:
predicted_label,correct_lablel = val_metrics(model1, data_loader_test, C=1000)

In [ ]:
binary_acc(predicted_label, correct_lablel)

In [ ]:
predicted_label1,correct_lablel1 = val_metrics(model1, data_loader_valid, C=1000)

In [ ]:
binary_acc(predicted_label1, correct_lablel1)

#take some image for test

In [ ]:
import random
labelsl = clase
    
# choose randomly five image to test our mpdel visualy
data_rot = '/content/drive/MyDrive/data/openimages_dataset/test/data'
image_name = os.listdir(data_rot) 
# pick randomly five image name from the test folder
test_image = random.sample(image_name,5)



In [ ]:
# choose a number between 0 to 4
b = 4 


#this calculate the image classes
nnn = Test[Test['ImageID']==test_image[b][:-4]]['LabelName']
images_classe = []
for ele in nnn:
  images_classe.append(Test_classes[Test_classes['classID']==ele]['classname'].iloc[0])
img = Image.open(os.path.join(data_rot,test_image[b])).convert("RGB")
img = img.resize((224,224))
transformTotensor = T1.Compose([T1.ToTensor()])
img1 = transformTotensor(img)
img1 = img1.unsqueeze(0)
img1 = img1.to(device)

loss = model1(img1)

try :
  labelssss = loss[0]['labels'][0]
  
except:
  labelssss = 0

image = Chitra(img, bboxes=loss[0]['boxes'][0].cpu().detach().numpy(), labels=labelsl[labelssss])
plt.imshow(image.draw_boxes())

print('the labels provided by our model are')
for i in loss[0]['labels'][:]:
    print(labelsl[i])
    print('-----------------')

print('the real labels ')
for i in images_classe:
    print(i)
    print('-----------------')
